# 各種インポート

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install -q timm

     |████████████████████████████████| 509 kB 4.3 MB/s 


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
import timm

# データとモデルの読み込み

In [ ]:
test_images = np.load('/content/drive/MyDrive/Competition/ProbSpace/日本画/data/test_data.npz')['arr_0']
test_images = test_images.transpose((0,3,1,2))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(test_images).float())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, num_workers=2, pin_memory=True)

In [ ]:
#model_name = 'resnet34'
#model_name = 'efficientnet_b3'
model_name = 'tf_efficientnet_b3_ns'
num_classes = 4
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold1*0728.pth')
#weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold2*0729.pth')
#weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold0_best0?.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold0_best_update.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold2_best0?.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold3_best_update.pth')
#weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold4_best_update.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold4_best0?.pth')
#weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/eff_b3_FocalLoss_weight_fold0*')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
weights

['/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold1_best0728.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold0_best_update.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold2_best02.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold3_best_update.pth',
 '/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_checkpoints/fold4_best03.pth']

In [ ]:
#model_name = 'resnet34'
#model_name = 'efficientnet_b3'
model_name = 'tf_efficientnet_b3_ns'
num_classes = 4
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold1*0728.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/TF_EfficientNetB3_NS/fold2*0729.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold0_best_update.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold3_best_update.pth')
weights += glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/Best_update/fold4_best_update.pth')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# 推論

In [ ]:
model_name = 'efficientnetv2_rw_s'

num_classes = 4
weights = glob.glob('/content/drive/MyDrive/Competition/ProbSpace/日本画/checkpoints/effv2_true/fold_all_best03.pth')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
all_predicts = np.zeros((2000, 4))
with torch.no_grad():
  for w in weights:
    model = timm.create_model(model_name, pretrained=False, num_classes=num_classes)
    model.load_state_dict(torch.load(w))
    mode = model.to(device)
    model.eval()
    predicts = []
    for x in test_loader:
      x = x[0].to(device)
      outputs = model(x)
      #preds = torch.max(outputs, dim=1)[1]
      predicts.append(outputs.cpu().numpy())
    all_predicts += np.concatenate(predicts)

In [ ]:
print(all_predicts.shape)
results = np.argmax(all_predicts, axis=1)
results.shape

(2000, 4)


(2000,)

In [ ]:
sub = pd.DataFrame(results, columns=['y'])
sub.index.name = 'id'
sub.index += 1

In [ ]:
sub.to_csv('/content/drive/MyDrive/Competition/ProbSpace/日本画/submission/submission_takasan_effv2_fold_all.csv')